# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation


10 countries with the lowest infant mortality rates

In [5]:
document = ET.parse( './data/mondial_database.xml' )

In [6]:
import pandas as pd
import numpy as np

In [7]:
data = {'country':[], 'infant_mortality':[]}

#Get the infant mortality of all countries
for element in document.iterfind('country'):
    data['country'].append(element.find('name').text)
    mortality = element.find('infant_mortality')
    if mortality is not None:
        data['infant_mortality'].append(mortality.text)
    else:
        data['infant_mortality'].append('')

In [8]:
df = pd.DataFrame(data)

# clean up countries with missing infant_mortality rates
df = df.replace('',np.NaN)

# find the 10 lowest infant mortality rates
df.dropna().sort(columns='infant_mortality').head(10)

/Users/rlaverack/ipykernel/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,country,infant_mortality
38,Monaco,1.81
30,Romania,10.16
153,Fiji,10.2
69,Brunei,10.48
132,Grenada,10.5
237,Mauritius,10.59
124,Panama,10.7
243,Seychelles,10.77
102,United Arab Emirates,10.92
113,Barbados,10.93


10 cities with the largest population

In [10]:
document = ET.parse( './data/mondial_database.xml' )
data = {'city':[], 'country':[], 'year':[], 'population':[]}

In [11]:
for city in document.findall('country/city'):
    country = city.attrib['country']
    name = ''
    population = 0
    year = 0
    for node in list(city):
        if node.tag == 'name':
            name = node.text
        elif node.tag == 'population':
            # note: if multiple population tags, the last is used
            # population tags sorted by year, so last is most recent
            population = int(node.text)
            year = int(node.attrib['year'])
    
    # only add cities with known populations
    if population != 0:
        data['country'].append(country)
        data['city'].append(name)
        data['population'].append(population)
        data['year'].append(year)

In [12]:
df = pd.DataFrame(data)

# find the 10 largest cities, by population
df.sort(columns='population', ascending=False).head(10)

/Users/rlaverack/ipykernel/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,city,country,population,year
165,Seoul,ROK,9708483,2010
154,Cairo,ET,8471859,2006
75,Bangkok,THA,7506700,1999
123,Hong Kong,HONX,7055071,2009
87,Ho Chi Minh City,VN,5968384,2009
201,Singapore,SGP,5076700,2010
153,Alexandria,ET,4123869,2006
205,New Taipei,RC,3939305,2012
166,Pusan,ROK,3403135,2010
102,Pyongyang,NOK,3255288,2008


10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [15]:
document = ET.parse( './data/mondial_database.xml' )
data = {'group':[], 'country':[], 'population':[]}

In [17]:
for country in document.findall('country'):
    name = ''
    group = ''
    population = 0
    for node in list(country):
        if node.tag == 'name':
            name = node.text
        elif node.tag == 'population':
            # note: if multiple population tags, the last is used
            # population tags sorted by year, so last is most recent
            population = int(node.text)
            year = int(node.attrib['year'])
        elif node.tag == 'ethnicgroup':
            percentage = float(node.attrib['percentage'])
            group = node.text
            
            if group and percentage and population:
            # only add groups with known populations
                data['country'].append(name)
                data['group'].append(group)
                # calculate ethnic group population from percentage of overall population
                data['population'].append(int(population * percentage / 100.))

In [18]:
df = pd.DataFrame(data)

total = df.groupby('group')[['population']].sum()

# find the 10 largest ethnic groups, by population
total.sort(columns='population', ascending=False).head(10)

/Users/rlaverack/ipykernel/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,population
group,
Han Chinese,1245058800
Indo-Aryan,871815583
European,494872201
African,318325104
Dravidian,302713744
Mestizo,157734349
Bengali,146776916
Russian,131856989
Japanese,126534212


name and country of a) longest river, b) largest lake and c) airport at highest elevation